<a href="https://colab.research.google.com/github/ychoi-kr/chatgpt-langchain/blob/main/chapter3/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ChatGPT를 API에서 이용하기

## 3-3 입력과 출력의 길이 제한 및 요금에 영향을 주는 '토큰'

### Tokenizer와 tiktoken 소개

In [1]:
!pip install tiktoken==0.6.0

In [2]:
import tiktoken

text = "It’s easy to make something cool with LLMs, but very hard to make something production-ready with them."

encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
tokens = encoding.encode(text)
print(len(tokens))

23


In [3]:
text = "LLM을 사용해서 멋져 보이는 것을 만들기는 쉽지만, 프로덕션 수준으로 만들어 내기는 매우 어렵다."

encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
tokens = encoding.encode(text)
print(len(tokens))

47


## 3-4 Chat Completions API를 접하는 환경 준비

### 구글 코랩 노트북 생성

In [4]:
print("Hello World")

Hello World


### OpenAI API 키 준비

In [5]:
import os

os.environ["OPENAI_API_KEY"] = "your-openai-api-key"

In [6]:
# 구글 코랩 보안 비밀에 API 키를 등록한 경우 다음 코드로 입력 가능
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

## 3-5 Chat Completions API 사용해 보기

### OpenAI 라이브러리

In [7]:
!pip install openai==1.14.3

### Chat Completions API 호출

In [8]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello! I'm John."}
  ]
)

print(response)

ChatCompletion(id='chatcmpl-99STGTd8g2zQZcasTHE1j13OeGzXW', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello John! How can I assist you today?', role='assistant', function_call=None, tool_calls=None))], created=1712041422, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint='fp_b28b39ffa8', usage=CompletionUsage(completion_tokens=10, prompt_tokens=23, total_tokens=33))


In [9]:
print(response.model_dump_json(indent=2))

{
  "id": "chatcmpl-99STGTd8g2zQZcasTHE1j13OeGzXW",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "Hello John! How can I assist you today?",
        "role": "assistant",
        "function_call": null,
        "tool_calls": null
      }
    }
  ],
  "created": 1712041422,
  "model": "gpt-3.5-turbo-0125",
  "object": "chat.completion",
  "system_fingerprint": "fp_b28b39ffa8",
  "usage": {
    "completion_tokens": 10,
    "prompt_tokens": 23,
    "total_tokens": 33
  }
}


### 대화 이력을 바탕으로 한 응답 얻기

In [10]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Hello! I'm John."},
        {"role": "assistant", "content": "Hello John! How can I assist you today?"},
        {"role": "user", "content": "Do you know my name?"}
    ]
)

print(response.model_dump_json(indent=2))

{
  "id": "chatcmpl-99STHS5QX6wrcYokbrEAP4QUxfDjo",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "Yes, you mentioned your name is John. How can I help you further, John?",
        "role": "assistant",
        "function_call": null,
        "tool_calls": null
      }
    }
  ],
  "created": 1712041423,
  "model": "gpt-3.5-turbo-0125",
  "object": "chat.completion",
  "system_fingerprint": "fp_b28b39ffa8",
  "usage": {
    "completion_tokens": 18,
    "prompt_tokens": 47,
    "total_tokens": 65
  }
}


### 응답을 스트리밍으로 받기

In [11]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Hello! I'm John."}
  ],
  stream=True
)

for chunk in response:
  choice = chunk.choices[0]
  if choice.finish_reason is None:
    print(choice.delta.content)


Hello
 John
!
 How
 can
 I
 assist
 you
 today
?


### （칼럼）Completions API

In [12]:
from openai import OpenAI

client = OpenAI()

response = client.completions.create(
  model="gpt-3.5-turbo-instruct",
  prompt="Hello! I'm John."
)

print(response.model_dump_json(indent=2))

{
  "id": "cmpl-99STJsiIW6XCmRm75Dysa1YPjVyip",
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": " I am an AI designed to assist you with various tasks and provide helpful information.\n\n"
    }
  ],
  "created": 1712041425,
  "model": "gpt-3.5-turbo-instruct",
  "object": "text_completion",
  "system_fingerprint": null,
  "usage": {
    "completion_tokens": 16,
    "prompt_tokens": 6,
    "total_tokens": 22
  }
}


In [13]:
prompt = """Human: Hello! I'm John.
AI: Nice to meet you, John!
Human: Do you know my name?
AI: """

response = client.completions.create(
  model="gpt-3.5-turbo-instruct",
  prompt=prompt
)
print(response.model_dump_json(indent=2))

{
  "id": "cmpl-99STJnTZXHlu17pWqJKaHCgemI8cP",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "Yes, your name is John."
    }
  ],
  "created": 1712041425,
  "model": "gpt-3.5-turbo-instruct",
  "object": "text_completion",
  "system_fingerprint": null,
  "usage": {
    "completion_tokens": 7,
    "prompt_tokens": 28,
    "total_tokens": 35
  }
}


## 3-6 Function calling

### Function calling의 샘플 코드

[OpenAI 공식 문서](https://platform.openai.com/docs/guides/gpt/function-calling)를 바탕으로 일부 수정한 코드입니다.

In [14]:
import json

def get_current_weather(location, unit="celsius"):
    weather_info = {
        "location": location,
        "temperature": "25",
        "unit": "celsius",
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

In [15]:
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. Seoul",
                },
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["location"],
        },
    }
]

In [16]:
messages = [{"role": "user", "content": "What's the weather like in Seoul?"}]

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
    functions=functions
)

print(response.model_dump_json(indent=2))

{
  "id": "chatcmpl-99STKrlHdZRUrKHsoM60Tv3Z6k0Gu",
  "choices": [
    {
      "finish_reason": "function_call",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": null,
        "role": "assistant",
        "function_call": {
          "arguments": "{\"location\":\"Seoul\"}",
          "name": "get_current_weather"
        },
        "tool_calls": null
      }
    }
  ],
  "created": 1712041426,
  "model": "gpt-3.5-turbo-0125",
  "object": "chat.completion",
  "system_fingerprint": "fp_b28b39ffa8",
  "usage": {
    "completion_tokens": 16,
    "prompt_tokens": 79,
    "total_tokens": 95
  }
}


In [17]:
response_message = response.choices[0].message

available_functions = {
    "get_current_weather": get_current_weather,
}
function_name = response_message.function_call.name
fuction_to_call = available_functions[function_name]
function_args = json.loads(response_message.function_call.arguments)

function_response = fuction_to_call(
    location=function_args.get("location"),
    unit=function_args.get("unit"),
)

print(function_response)

{"location": "Seoul", "temperature": "25", "unit": "celsius", "forecast": ["sunny", "windy"]}


In [18]:
messages.append(response_message)
messages.append(
    {
        "role": "function",
        "name": function_name,
        "content": function_response,
    }
)

In [19]:
print(messages)

[{'role': 'user', 'content': "What's the weather like in Seoul?"}, ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"location":"Seoul"}', name='get_current_weather'), tool_calls=None), {'role': 'function', 'name': 'get_current_weather', 'content': '{"location": "Seoul", "temperature": "25", "unit": "celsius", "forecast": ["sunny", "windy"]}'}]


In [20]:
second_response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
)

print(second_response.model_dump_json(indent=2))

{
  "id": "chatcmpl-99STLIiqvY9XtkSI8igYDZGwq1J52",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The weather in Seoul is currently 25°C and sunny with windy conditions.",
        "role": "assistant",
        "function_call": null,
        "tool_calls": null
      }
    }
  ],
  "created": 1712041427,
  "model": "gpt-3.5-turbo-0125",
  "object": "chat.completion",
  "system_fingerprint": "fp_b28b39ffa8",
  "usage": {
    "completion_tokens": 15,
    "prompt_tokens": 71,
    "total_tokens": 86
  }
}
